In [1]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler

def extract_features(file_path, max_length=1000):
    audio, sr = librosa.load(file_path, mono=True)

    # 1. Coeficientes MFCC
    mfccs = librosa.feature.mfcc(y=audio, sr=sr)
    mfcc_features = np.concatenate((mfccs.mean(axis=1), mfccs.std(axis=1)))

    # 2. Cromagrama
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)

    # 3. Contraste espectral
    contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)

    # 4. Tonnetz
    tonnetz = librosa.feature.tonnetz(y=audio, sr=sr)

    # 5. Tempograma
    tempo, tempogram = librosa.beat.beat_track(y=audio, sr=sr)

    # Asegurar que todas las características tengan la misma longitud
    all_features = np.concatenate((mfcc_features, chroma.mean(axis=1)[:max_length], contrast.mean(axis=1)[:max_length], tonnetz.mean(axis=1)[:max_length], tempogram[:max_length]))
    
    # Rellenar con ceros si es necesario
    if len(all_features) < max_length:
        all_features = np.pad(all_features, (0, max_length - len(all_features)))

    return all_features

def save_features(folder_path):
    features = []
    names=[]
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".mp3"):
            file_path = os.path.join(folder_path, file_name)
            song_features = extract_features(file_path)
            features.append(song_features)
            names.append(file_name)
    return features, names
    #np.save(output_file, np.array(features))
    
folder_path = 'es'
caracteristicas, nombres= save_features(folder_path)


In [7]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler


caracteristicas = np.array(caracteristicas)

# Cargar características existentes
print("-----------CARGADO------------")
# Configurar el modelo k-NN
n_neighbors = 4
knn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')

# Ajustar y transformar las características existentes con StandardScaler
scaler = StandardScaler()
existing_features_scaled = scaler.fit_transform(caracteristicas)
knn_model.fit(existing_features_scaled)

# Calcular características de la nueva canción y transformar con el mismo StandardScaler
nueva_cancion_features = extract_features("Bad Bunny - NI BIEN NI MAL.mp3")
nueva_cancion = scaler.transform([nueva_cancion_features])

# Encontrar vecinos cercanos
#distances, indices = knn_model.kneighbors(nueva_cancion)
nueva_cancion_vecinos = knn_model.kneighbors(nueva_cancion)[1]

#print(f"Vecinos cercanos para la nueva canción: {indices} con distancia {distances}")


# Imprimir los nombres de las canciones más similares
print("Canciones más similares:")
for vecino_idx in nueva_cancion_vecinos[0]:
    print(nombres[vecino_idx])

"""
umbral_similitud = 0.9

# Filtrar vecinos cercanos basados en el umbral de similitud
vecinos_filtrados = [(indice, distancia) for indice, distancia in zip(indices[0], distances[0]) if distancia <= umbral_similitud]

print(f"Vecinos cercanos para la nueva canción (distancia <= {umbral_similitud}): {vecinos_filtrados}")"""

-----------CARGADO------------
Canciones más similares:
Bad Bunny - NI BIEN NI MAL.mp3
Noriel, Myke Towers, Rauw Alejandro, Almighty - Cuerpo en Venta.mp3
DrefQuila - Ella busca.mp3
Don Omar - Pobre Diabla.mp3


'\numbral_similitud = 0.9\n\n# Filtrar vecinos cercanos basados en el umbral de similitud\nvecinos_filtrados = [(indice, distancia) for indice, distancia in zip(indices[0], distances[0]) if distancia <= umbral_similitud]\n\nprint(f"Vecinos cercanos para la nueva canción (distancia <= {umbral_similitud}): {vecinos_filtrados}")'

### KNN queque


In [8]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from queue import PriorityQueue

#caracteristicas = np.array(caracteristicas)
nueva_cancion_trans = np.array(nueva_cancion)

In [9]:
def knn_search_priority_queue(query, k):
    distances = euclidean_distances(query.reshape(1, -1), caracteristicas)[0]
    priority_queue = PriorityQueue()
    for i, dist in enumerate(distances):
        priority_queue.put((dist, nombres[i]))
    neighbors = []
    for _ in range(k):
        dist, neighbor = priority_queue.get()
        neighbors.append((neighbor, dist))

    return neighbors

In [13]:
def range_search(query, radius):
    distances = euclidean_distances(query.reshape(1, -1), caracteristicas)[0]
    result = [(nombres[i], dist) for i, dist in enumerate(distances) if dist <= radius]
    return result

In [22]:
k = 5
radius_values = [100]

# Búsqueda KNN con cola de prioridad
knn_result_priority_queue = knn_search_priority_queue(nueva_cancion_features, k)
print(f"Vecinos más cercanos (KNN con cola de prioridad): {knn_result_priority_queue}")


Vecinos más cercanos (KNN con cola de prioridad): [('Bad Bunny - NI BIEN NI MAL.mp3', 0.0), ('Oz - Espero Que Entiendas.mp3', 31093.11309036824), ('Bonka - Hoy.mp3', 33536.24875223837), ('DrefQuila - Ella busca.mp3', 38152.24018110978), ('Enrique Iglesias, Descemer Bueno, Gente De Zona - Bailando - Spanish Version.mp3', 38689.007864860476)]


In [23]:
# Búsqueda por rango
for radius in radius_values:
    range_result = range_search(nueva_cancion_features, radius)
    print(f"Resultados para rango {radius}: {range_result}")

Resultados para rango 100: [('Bad Bunny - NI BIEN NI MAL.mp3', 0.0)]


In [30]:
import numpy as np

def knn_search_sequential(query, k, features, names):
    distances = []
    for i, feature in enumerate(features):
        dist = np.linalg.norm(query - feature)
        distances.append((dist, names[i]))

    distances.sort(key=lambda x: x[0])  # Ordenar por distancia
    neighbors = distances[:k]
    return neighbors

# Suponiendo que `caracteristicas` es tu conjunto de características y `nombres` son los nombres correspondientes
# Asegúrate de tener definidos estos datos antes de llamar a la función
#nueva_cancion_features = extract_features("Bad Bunny - NI BIEN NI MAL.mp3")
#caracteristicas = np.array(caracteristicas)
#nombres = np.array(nombres)

# Búsqueda KNN secuencial
k = 5
knn_result_sequential = knn_search_sequential(nueva_cancion_trans, k, caracteristicas, nombres)
print(f"Vecinos más cercanos (KNN secuencial): {knn_result_sequential}")


Vecinos más cercanos (KNN secuencial): [(58348.83726952062, 'Lunay - Fin De Semana.mp3'), (68871.22173929434, 'Cypress Hill - Marijuano Locos (Stoned Raiders) - Spanish Version.mp3'), (68999.48659666223, 'J Balvin - Ginza.mp3'), (69545.6553076493, 'Reik, Manuel Turizo - Aleluya.mp3'), (70271.46177208882, 'YSY A - ALMA (prod. 0-600).mp3')]


In [46]:
def distancia_euclidiana(vector1, vector2):
    suma_cuadrados = 0
    for p1, p2 in zip(vector1, vector2):
        suma_cuadrados += (p1 - p2) ** 2
    distance = suma_cuadrados ** 0.5
    return distance

def knnSearch(Collection, Query, k):
    result = []
    for i, vector in enumerate(Collection):
        dist = distancia_euclidiana(Query, vector)
        result.append((i, dist))
    result.sort(key=lambda tup: tup[1])
    return result[:k]

aa= nueva_cancion_features
# Supongamos que 'caracteristicas' es tu ndarray y 'nueva_cancion_features' es el vector de características de la nueva canción
nueva_cancion_vecinos_secuencial = knnSearch(caracteristicas, nueva_cancion_features, k)

#caracteristicas = np.array(caracteristicas)
#nueva_cancion_trans = np.array(nueva_cancion)

# Imprimir los resultados
print("Vecinos más cercanos (KNN secuencial):", nueva_cancion_vecinos_secuencial)
for indice, valor in nueva_cancion_vecinos_secuencial:
    print(nombres[indice])

Vecinos más cercanos (KNN secuencial): [(7, 0.0), (71, 31093.113090368337), (10, 33536.24875223839), (30, 38152.24018110975), (37, 38689.00786486043)]
Bad Bunny - NI BIEN NI MAL.mp3
Oz - Espero Que Entiendas.mp3
Bonka - Hoy.mp3
DrefQuila - Ella busca.mp3
Enrique Iglesias, Descemer Bueno, Gente De Zona - Bailando - Spanish Version.mp3
